Stacking of SDSS QSO in the XXL-North field using RapidXMM
=================================

We do stacking for SDSS QSO far from any 4XMM sources (> 1 arcmin), and at random positions in the field.

We first stack QSO with no X-ray counterpart:

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from astropy.table import Table, unique

data_folder = Path("./data")
qso_filename = data_folder.joinpath("uls_qso_farfrom_xsources_xxl.fits")
qso = Table.read(qso_filename)
qso = unique(qso, keys="npixel")
len(qso)

In [ ]:
from api.stacking import stacking

_, _, _, _, _, scale, n, nsrc = stacking.stack_npixels(
    qso["npixel"],
    level_neighbours=6,
    params=None,
    max_data=10000,
    calc_flux=False,
    use_flagged_pixels=False,
    custom_bkg=True,
    with_plots=True,
    plotfile=Path("stacking_qso_xxl.pdf")
)

Now we stack random positions in the XXL field selected far from any X-ray source or optical QSO. For this we have defined a MOC where areas within 30 arcsec around any QSO or X-ray sources are removed. We select random healpix cells within that MOC for stacking.

In [ ]:
import numpy as np
from mocpy import MOC
from mocpy.core import flatten_pixels

xxl_masked_moc_file = data_folder.joinpath("xxl_sdssqso_4xmm_masked.moc")
xxl_masked_moc = MOC.from_fits(xxl_masked_moc_file)
xxl_masked_hpixels = flatten_pixels(xxl_masked_moc._interval_set._intervals, 16)

rng = np.random.default_rng()
qsornd = rng.choice(xxl_masked_hpixels, size=nsrc, replace=False)
qsornd = qsornd.astype(int)

In [ ]:
stacking.stack_npixels(
    qsornd,
    level_neighbours=6,
    params=None,
    max_data=10000,
    calc_flux=False,
    use_flagged_pixels=False,
    custom_bkg=True,
    with_plots=True,
    plotfile=Path("stacking_rnd_xxl.pdf"),
    scale=scale,
)